# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# Importing data

In [27]:
path=r"C:\Users\chenh\z_Careerfoundry\A4_Instacart Basket Analysis"
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)
df_ords = pd.read_csv(os.path.join(path,'02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

# Data Consistency Checks

## Mixed-Type Data

In [28]:
#we will practice fixing mixed-type data by creating a small test dataframe
df_test = pd.DataFrame()
df_test['mix'] = ['a', 'b', 1, True]

In [29]:
# Check for any mixed-type columns
for col in df_test.columns.tolist():
    weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_test[weird]) > 0:
        print (col)

mix


In [30]:
# Convert column's data from numeric to string
df_test['mix']=df_test['mix'].astype('str')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mix     4 non-null      object
dtypes: object(1)
memory usage: 160.0+ bytes


##  Missing Values

In [31]:
#Run a describe on df_prods to check the values
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


The max value of prices is 99,999, which requires further investigation to determine if it is an outlier.

In [32]:
df_prods[df_prods['prices']==99999]

,product_id,product_name,aisle_id,department_id,prices
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


In [34]:
df_prods[df_prods['product_name'].str.strip().str.replace(' ', '') == '2%ReducedFatMilk']

,product_id,product_name,aisle_id,department_id,prices
23910,23909,2% Reduced Fat Milk,84,16,9.2
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


In [35]:
df_prods.loc[df_prods['product_id'] == 33664, 'prices'] = 9.2

In [36]:
df_prods[df_prods['product_name'].str.strip().str.replace(' ', '') == '2%ReducedFatMilk']

,product_id,product_name,aisle_id,department_id,prices
23910,23909,2% Reduced Fat Milk,84,16,9.2
33666,33664,2 % Reduced Fat Milk,84,16,9.2


In [37]:
#Run a describe to check
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,7.981985
std,14343.717401,38.316774,5.850282,66.937784
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,14900.000000


The max value of prices is 14900, which requires further investigation to determine if it is an outlier.

In [38]:
df_prods[df_prods['prices']==14900]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0


In [39]:
df_prods[df_prods['product_name'].str.strip().str.replace(' ', '') == 'Lowfat2%MilkfatCottageCheese']

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0


In [40]:
#There are no identical products, replace the 'price' with the average value.
df_prods.loc[df_prods['product_id'] == 21553, 'prices'] = 7.98

In [41]:
#Run a describe to check
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,7.682305
std,14343.717401,38.316774,5.850282,4.200163
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,25.000000


In [42]:
# Find missing values (takes sum of Trues in a  column)
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [43]:
# Create a subset containing only missing values
df_nan=df_prods[df_prods['product_name'].isnull()==True]

In [44]:
# Create a subset containing only non-missing values
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [45]:
df_prods.shape

(49693, 5)

In [46]:
df_prods_clean.shape

(49677, 5)

## Duplicate Values

In [47]:
# Create subset of df_prods_clean that contains only rows of duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [48]:
# Dropping the duplicate values by creating a new dataframe
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [49]:
df_prods_clean_no_dups.shape

(49672, 5)

# Task

## Step 2

In [50]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


### There are many missing values in the 'days_since_prior_order' column, as its count is lower than the other columns´

###  'order_number' has min value as 1 and max value as 100.  No discrepancies found.

###  'days_since_prior_order' has min value as 0 and max as 30. No discrepancies found.

## Step 3

In [51]:
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords[weird]) > 0:
        print (col)
    else :
        print (col+ "does not have mixed-type data")

Unnamed: 0does not have mixed-type data
order_iddoes not have mixed-type data
user_iddoes not have mixed-type data
order_numberdoes not have mixed-type data
orders_day_of_weekdoes not have mixed-type data
order_hour_of_daydoes not have mixed-type data
days_since_prior_orderdoes not have mixed-type data


## Step 5

In [52]:
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

### There are 206,209 missing values in the 'days_since_prior_order' column. These are probably first orders placed by customers.

## Step6

### Removing the missing values would result in the loss of important information about customers, therefore I will create a new variable that acts like a flag based on the missing value.

In [53]:
df_ords['First_Buy']=df_ords['days_since_prior_order'].isnull()

In [54]:
df_ords.head()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,First_Buy
0,0,2539329,1,1,2,8,NaN,True
1,1,2398795,1,2,3,7,15.0,False
2,2,473747,1,3,3,12,21.0,False
3,3,2254736,1,4,4,7,29.0,False
4,4,431534,1,5,4,15,28.0,False


## Step7

In [55]:
df_ords.duplicated().sum()

0

### No duplicates found.

## Step 9

In [56]:
# Exporting df_prods_clean_no_dups dataframe as "products_checked.csv"
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'))

In [57]:
# Exporting df_ords dataframe as "orders_checked.csv"
df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'))